In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn import preprocessing
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### Indicator 1 - Renters who pay more than 50% of their income on housing

In [11]:
rent = pd.read_csv('../data/rent-as-percentage-income/ACSDT5Y2018.B25070_data_with_overlays_2020-11-19T201947.csv',
                   low_memory=False, skiprows=[1],
                   usecols=['NAME','B25070_010E','B25070_001E','B25070_011E'])
rent.rename(columns={'B25070_010E': 'paying_50p_more', 'B25070_001E': 'Total',
                    'B25070_011E': 'Total_not_comp'}, inplace = True)

rent.head()

,NAME,Total,paying_50p_more,Total_not_comp
0,"Census Tract 37, New York County, New York",789,127,61
1,"Census Tract 86.01, New York County, New York",1001,204,49
2,"Census Tract 44, New York County, New York",8145,1439,224
3,"Census Tract 86.03, New York County, New York",736,204,16
4,"Census Tract 86.02, New York County, New York",0,0,0


In [12]:
rent['pay over 50'] = (rent['paying_50p_more'] / (rent['Total'] - rent['Total_not_comp'])).round(decimals=2)
rent

,NAME,Total,paying_50p_more,Total_not_comp,pay over 50
0,"Census Tract 37, New York County, New York",789,127,61,0.17
1,"Census Tract 86.01, New York County, New York",1001,204,49,0.21
2,"Census Tract 44, New York County, New York",8145,1439,224,0.18
3,"Census Tract 86.03, New York County, New York",736,204,16,0.28
4,"Census Tract 86.02, New York County, New York",0,0,0,NaN
...,...,...,...,...,...
2162,"Census Tract 170.12, Richmond County, New York",324,46,106,0.21
2163,"Census Tract 244.02, Richmond County, New York",345,84,33,0.27
2164,"Census Tract 277.05, Richmond County, New York",470,159,66,0.39
2165,"Census Tract 277.06, Richmond County, New York",275,60,59,0.28


In [51]:
# # rows with NaN - because has 0's for the columns being used for calculations
# rent[rent.isna().any(axis=1)]

# # rent table without  NaN 

# rent.dropna()

### Indicator 2 - Amount of renters who have too little left ($12K) after paying housing cost

In [35]:
income = pd.read_csv('../data/low-income/ACSDT5Y2018.B25074_data_with_overlays_2020-12-10T140704.csv',
                   low_memory=False,
                     usecols=['NAME','B25074_001E','B25074_002E','B25074_013E','B25074_014E',
                             'B25074_015E','B25074_016E','B25074_017E','B25074_018E',
                             'B25074_027E'])
income.head(3)

,NAME,B25074_001E,B25074_002E,B25074_013E,B25074_014E,B25074_015E,B25074_016E,B25074_017E,B25074_018E,B25074_027E
0,Geographic Area Name,Estimate!!Total,"Estimate!!Total!!Less than $10,000","Estimate!!Total!!$10,000 to $19,999!!20.0 to 2...","Estimate!!Total!!$10,000 to $19,999!!25.0 to 2...","Estimate!!Total!!$10,000 to $19,999!!30.0 to 3...","Estimate!!Total!!$10,000 to $19,999!!35.0 to 3...","Estimate!!Total!!$10,000 to $19,999!!40.0 to 4...","Estimate!!Total!!$10,000 to $19,999!!50.0 perc...","Estimate!!Total!!$20,000 to $34,999!!50.0 perc..."
1,"Census Tract 429.02, Bronx County, New York",1442,178,0,0,14,0,0,322,158
2,"Census Tract 330, Bronx County, New York",2024,215,19,34,19,57,80,287,133


In [36]:
#dropping first row and renaming columns
income.columns = ['census-tract','total-renters','income-less-10k','income-15k-B25074_013E','income-15k-B25074_014E',
                             'income-15k-B25074_015E','income-15k-B25074_016E','income-15k-B25074_017E','income-15k-B25074_018E',
                             'income-27.5k-B25074_027E']
income = income.drop(index = 0, axis = 0)
income.head(3)

,census-tract,total-renters,income-less-10k,income-15k-B25074_013E,income-15k-B25074_014E,income-15k-B25074_015E,income-15k-B25074_016E,income-15k-B25074_017E,income-15k-B25074_018E,income-27.5k-B25074_027E
1,"Census Tract 429.02, Bronx County, New York",1442,178,0,0,14,0,0,322,158
2,"Census Tract 330, Bronx County, New York",2024,215,19,34,19,57,80,287,133
3,"Census Tract 358, Bronx County, New York",834,0,0,0,0,0,0,99,98


In [37]:
# total amount of renters with less than 12k after paying rent

income['total-renters-less12k']= income[['income-less-10k','income-15k-B25074_013E','income-15k-B25074_014E',
                             'income-15k-B25074_015E','income-15k-B25074_016E','income-15k-B25074_017E','income-15k-B25074_018E',
                             'income-27.5k-B25074_027E']].sum(axis =1)
income[['census-tract','total-renters','total-renters-less12k']].head()

,census-tract,total-renters,total-renters-less12k
1,"Census Tract 429.02, Bronx County, New York",1442,1.780014e+14
2,"Census Tract 330, Bronx County, New York",2024,2.151934e+18
3,"Census Tract 358, Bronx County, New York",834,9.998000e+03
4,"Census Tract 371, Bronx County, New York",1558,2.353769e+18
5,"Census Tract 385, Bronx County, New York",1648,4.210503e+16


In [38]:
#as a portion of total renters

income['percentage-renters-less12k'] = income['total-renters-less12k'].astype(float) / income['total-renters'].astype(float)

income[['census-tract','total-renters','total-renters-less12k','percentage-renters-less12k' ]].head()

,census-tract,total-renters,total-renters-less12k,percentage-renters-less12k
1,"Census Tract 429.02, Bronx County, New York",1442,1.780014e+14,1.234406e+11
2,"Census Tract 330, Bronx County, New York",2024,2.151934e+18,1.063209e+15
3,"Census Tract 358, Bronx County, New York",834,9.998000e+03,1.198801e+01
4,"Census Tract 371, Bronx County, New York",1558,2.353769e+18,1.510763e+15
5,"Census Tract 385, Bronx County, New York",1648,4.210503e+16,2.554917e+13


### Indicator 3 - UI coverage rate

#### Method
\begin{align*}
&\text{UI coverage rate} = \frac{\text{private-sector workers in the UI program}}{\text{number of workers in the private for-profit and nonprofit sectors}}\\
\\
&\text{non-UI coverage rate} = 1-\text{UI coverage rate}
\end{align*}

where the private-sector workers in the UI program comes from **Total number of jobs (C000)** in Private Jobs in  Longitudinal Employer–Household Dynamics (LEHD) for 2013‒17 (the five most recent years available), and the estimated labor force comes from **Private wage and salary workers (DP03_0047E)** in the corresponding 2013‒17 ACS. 

Data collection and cleanning details can be found in **UCLA_UI.ipnb**

In [13]:
acs = pd.read_csv('../data/UI_coverage/ACS_DP03_cleaned.csv', low_memory=False)
rac_avg = pd.read_csv('../data/UI_coverage/ny_rac_cleaned.csv', low_memory=False)

In [17]:
ui_coverage = acs.merge(rac_avg, left_on='GEO_ID', right_on='trct', how='left')
ui_coverage['ui_rate'] = ui_coverage['total_ui_jobs'] / ui_coverage['total_jobs']
ui_coverage = ui_coverage[['GEO_ID', 'NAME', 'total_ui_jobs', 'total_jobs', 'ui_rate']]

print('# ui rate>1:', sum(ui_coverage['ui_rate']>1))
print(ui_coverage[~np.isinf(ui_coverage.ui_rate)].ui_rate.describe())

# ui rate>1: 1016
count    2117.000000
mean        1.076616
std         0.913469
min         0.444891
25%         0.872669
50%         0.982034
75%         1.108605
max        31.350000
Name: ui_rate, dtype: float64


In [18]:
ui_coverage.loc[ui_coverage['ui_rate']>1, 'ui_rate'] = 1
ui_coverage.head()

,GEO_ID,NAME,total_ui_jobs,total_jobs,ui_rate
0,36061000100,"Census Tract 1, New York County, New York",175.20,0.0,1.0
1,36061000201,"Census Tract 2.01, New York County, New York",1087.80,892.0,1.0
2,36061000202,"Census Tract 2.02, New York County, New York",2427.25,2365.0,1.0
3,36061000500,"Census Tract 5, New York County, New York",167.35,0.0,1.0
4,36061000600,"Census Tract 6, New York County, New York",3531.35,2481.0,1.0


### Indicator 3 - Unemployment rate

In [23]:
employment = pd.read_csv('../data/employment_ACSDP03_2018.csv',
                         low_memory=False, skiprows=[1],
                         usecols=['GEO_ID','NAME','DP03_0003E','DP03_0005E','DP03_0037E'])

In [24]:
employment.head()

,GEO_ID,NAME,DP03_0003E,DP03_0005E,DP03_0037E
0,1400000US36005000100,"Census Tract 1, Bronx County, New York",0,0,0
1,1400000US36005000200,"Census Tract 2, Bronx County, New York",1873,295,57
2,1400000US36005000400,"Census Tract 4, Bronx County, New York",3052,244,207
3,1400000US36005001600,"Census Tract 16, Bronx County, New York",2477,164,306
4,1400000US36005001900,"Census Tract 19, Bronx County, New York",1451,192,103


In [25]:
employment['DP03_0003E']=employment['DP03_0003E'].astype(float)
employment['DP03_0005E']=employment['DP03_0005E'].astype(float)

In [26]:
employment['unemployment rate'] = (100 * employment['DP03_0005E'] / employment['DP03_0003E']).round(decimals=2)
employment.head()

,GEO_ID,NAME,DP03_0003E,DP03_0005E,DP03_0037E,unemployment rate
0,1400000US36005000100,"Census Tract 1, Bronx County, New York",0.0,0.0,0,NaN
1,1400000US36005000200,"Census Tract 2, Bronx County, New York",1873.0,295.0,57,15.75
2,1400000US36005000400,"Census Tract 4, Bronx County, New York",3052.0,244.0,207,7.99
3,1400000US36005001600,"Census Tract 16, Bronx County, New York",2477.0,164.0,306,6.62
4,1400000US36005001900,"Census Tract 19, Bronx County, New York",1451.0,192.0,103,13.23


### Indicator 4 - employment in impacted sectors (retail sales, hospitality service, personal care)